In [1]:
import tensorflow_hub as hub
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from python_scripts.read_xwav_header import read_xwav_header
import sys
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pickle

In [2]:
model = hub.load('https://www.kaggle.com/models/google/humpback-whale/frameworks/TensorFlow2/variations/humpback-whale/versions/1')

In [3]:
annot = pd.read_csv("data/pifsc_products_detections_annotations.csv")
annot.head()

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc
0,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1,True,True,Other,0.213,0.213,2005-06-06 13:00:00.213000+00:00,2005-06-06 13:00:00.213000+00:00
1,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,13,True,True,Other,0.108,0.108,2005-06-06 14:15:00.108000+00:00,2005-06-06 14:15:00.108000+00:00
2,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Other,0.143,0.143,2005-07-02 04:00:00.143000+00:00,2005-07-02 04:00:00.143000+00:00
3,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,17,True,True,Other,0.213,0.213,2005-07-02 05:15:00.213000+00:00,2005-07-02 05:15:00.213000+00:00
4,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,22,True,True,Other,0.213,0.213,2005-08-01 10:15:00.213000+00:00,2005-08-01 10:15:00.213000+00:00


In [4]:
annot["flac_compressed_xwav_object"].nunique()

5489

In [10]:
uncompressed_files = os.listdir("data/audio_data/uncompressed")
uncompressed_files

['Cross_A_01_050606_123845.d20.x.wav',
 'Cross_A_01_050702_031345.d20.x.wav',
 'Cross_A_01_050801_074615.d20.x.wav',
 'Cross_A_01_050801_110500.d20.x.wav',
 'Cross_A_01_050908_025000.d20.x.wav',
 'Equator_A_01_120313_165615.df20.x.wav',
 'Equator_A_01_120419_012845.df20.x.wav',
 'Hawaii_K_01_070813_065730.d20.x.wav',
 'Hawaii_K_01_070813_073500.d20.x.wav',
 'Hawaii_K_01_070915_204730.d20.x.wav',
 'Hawaii_K_01_070915_212500.d20.x.wav',
 'Hawaii_K_02_080506_173715.d20.x.wav',
 'Hawaii_K_02_080506_183545.d20.x.wav',
 'Hawaii_K_02_080609_205945.d20.x.wav',
 'Hawaii_K_02_080702_115545.d20.x.wav',
 'Hawaii_K_02_080702_125715.d20.x.wav',
 'Hawaii_K_03_080807_141730.d20.x.wav',
 'Hawaii_K_03_080807_161500.d20.x.wav',
 'Hawaii_K_03_080815_114500.d20.x.wav',
 'Hawaii_K_03_080921_234730.d20.x.wav',
 'Hawaii_K_03_080922_014500.d20.x.wav',
 'Hawaii_K_03_081010_164730.d20.x.wav',
 'Hawaii_K_06_090505_164500.d20.x.wav']

In [11]:
lst = []
for file in uncompressed_files:
    data = {}
    if "Equator" in file:
        continue
    file_path = f"data/audio_data/uncompressed/{file}"
    data["file_name"] = file
    data["file_path"] = file_path
    data["header"] = read_xwav_header(file_path)
    
    waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(file_path))
    
    w = tf.expand_dims(waveform, 0)
    del waveform
    pcen_spec = model.front_end(w)
    del w
    data["pcen_spec"] = pcen_spec
    lst.append(data)

In [12]:
with open("data/audio_data/processed/first_spcs.pkl", "wb") as f:
    pickle.dump(lst, f)

In [30]:
f = "data/audio_data/uncompressed/Cross_A_01_050801_110500.d20.x.wav"
waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(f))

In [37]:
waveform

<tf.Tensor: shape=(22509000, 1), dtype=float32, numpy=
array([[0.01321411],
       [0.00869751],
       [0.00982666],
       ...,
       [0.01013184],
       [0.01025391],
       [0.01025391]], dtype=float32)>

In [39]:
read_xwav_header("data/audio_data/uncompressed/Cross_A_01_050801_110500.d20.x.wav")

({'chunk_id': b'RIFF',
  'chunk_size': 45019060,
  'format': b'WAVE',
  'subchunk1_id': b'fmt ',
  'subchunk1_size': 16,
  'audio_format': 1,
  'num_channels': 1,
  'sample_rate': 10000,
  'byte_rate': 20000,
  'block_align': 2,
  'bits_per_sample': 16},
 {'harp_subchunk_id': b'harp',
  'harp_subchunk_size': 1016,
  'wav_version_number': 1,
  'firmware_version_number': b'1.17\x00\x00\x00\x00\x00\x00',
  'instrument_id': b'DL11',
  'site_name': b'XXXX',
  'experiment_name': b'Cross_01',
  'disk_sequence_number': 9,
  'disk_serial_number': b'12345678',
  'num_of_raw_files': 30,
  'longitude': -158.25383,
  'latitude': 18.72208,
  'depth': 398,
  'reserved': b'\x00\x00\x00\x00\x00\x00\x00\x00'},
 defaultdict(dict,
             {0: {'year': 5,
               'month': 8,
               'day': 1,
               'hour': 11,
               'minute': 5,
               'second': 0,
               'ticks': 0,
               'byte_loc': 1068,
               'byte_len': 1500000,
               'wri

In [6]:
with open("data/audio_data/processed/first_spcs.pkl", "rb") as f:
    data = pickle.load(f)

In [14]:
lst[0]

{'file_name': 'Cross_A_01_050606_123845.d20.x.wav',
 'file_path': 'data/audio_data/uncompressed/Cross_A_01_050606_123845.d20.x.wav',
 'header': ({'chunk_id': b'RIFF',
   'chunk_size': 45001060,
   'format': b'WAVE',
   'subchunk1_id': b'fmt ',
   'subchunk1_size': 16,
   'audio_format': 1,
   'num_channels': 1,
   'sample_rate': 10000,
   'byte_rate': 20000,
   'block_align': 2,
   'bits_per_sample': 16},
  {'harp_subchunk_id': b'harp',
   'harp_subchunk_size': 1016,
   'wav_version_number': 1,
   'firmware_version_number': b'1.17\x00\x00\x00\x00\x00\x00',
   'instrument_id': b'DL13',
   'site_name': b'CSM1',
   'experiment_name': b'Cross_01',
   'disk_sequence_number': 4,
   'disk_serial_number': b'12345678',
   'num_of_raw_files': 30,
   'longitude': -18.43343,
   'latitude': 158.15221,
   'depth': 396,
   'reserved': b'\x00\x00\x00\x00\x00\x00\x00\x00'},
  defaultdict(dict,
              {0: {'year': 5,
                'month': 6,
                'day': 6,
                'hour': 12

In [21]:
model.signatures

_SignatureMap({'score': <ConcreteFunction signature_wrapper(*, context_step_samples, waveform) at 0x22BDFDE4040>, 'metadata': <ConcreteFunction signature_wrapper() at 0x22BE8FF66E0>, 'serving_default': <ConcreteFunction signature_wrapper(*, context_step_samples, waveform) at 0x22BE65D0040>})

In [23]:
model.signatures['serving_default']

<ConcreteFunction signature_wrapper(*, context_step_samples, waveform) at 0x22BE65D0040>

In [24]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_checkpoint_dependencies',
 '_default_save_signature',
 '_deferred_dependencies',
 '_delete_tracking',
 '_deserialization_dependencies',
 '_deserialize_from_proto',
 '_export_to_saved_model_graph',
 '_gather_saveables_for_checkpoint',
 '_handle_deferred_dependencies',
 '_is_hub_module_v1',
 '_lookup_dependency',
 '_map_resources',
 '_maybe_initialize_trackable',
 '_name_based_attribute_restore',
 '_name_based_restores',
 '_no_dependency',
 '_object_identifier',
 '_pcen',
 '_preload_simple_restoration',
 '_restore_from_tensors',
 '_self_name_based_

In [38]:
model.front_end

TypeError: f(inputs, training, mask) missing required arguments: inputs.

In [27]:
dir(model.front_end)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_checkpoint_dependencies',
 '_deferred_dependencies',
 '_delete_tracking',
 '_deserialization_dependencies',
 '_deserialize_from_proto',
 '_export_to_saved_model_graph',
 '_gather_saveables_for_checkpoint',
 '_handle_deferred_dependencies',
 '_lookup_dependency',
 '_map_resources',
 '_maybe_initialize_trackable',
 '_name_based_attribute_restore',
 '_name_based_restores',
 '_no_dependency',
 '_object_identifier',
 '_preload_simple_restoration',
 '_restore_from_tensors',
 '_self_name_based_restores',
 '_self_saveable_object_factories',
 '_self_setat

In [44]:
model.front_end.variables

ListWrapper([<tf.Variable 'pcen/alpha:0' shape=(64,) dtype=float32, numpy=
array([0.96838385, 0.9761289 , 0.98089886, 0.97721404, 0.9871994 ,
       0.9845029 , 0.978106  , 0.97405255, 0.97512984, 0.981545  ,
       0.98054516, 0.98297554, 0.9833554 , 0.9846124 , 0.9814225 ,
       0.9747    , 0.97760785, 0.9796331 , 0.97838956, 0.9814383 ,
       0.9817132 , 0.97727287, 0.9809541 , 0.98087126, 0.9795548 ,
       0.9811239 , 0.97469366, 0.9782519 , 0.98233706, 0.9809858 ,
       0.97864866, 0.9840678 , 0.9811113 , 0.9819034 , 0.97856784,
       0.97762   , 0.9771842 , 0.9835421 , 0.98621875, 0.98015136,
       0.98395246, 0.9833393 , 0.9843758 , 0.9758238 , 0.9792226 ,
       0.9836882 , 0.9808793 , 0.98054606, 0.977489  , 0.98499143,
       0.97983825, 0.9762578 , 0.9835462 , 0.9800017 , 0.9793947 ,
       0.97670066, 0.9769266 , 0.9834014 , 0.9769312 , 0.9772469 ,
       0.9790105 , 0.9817227 , 0.9759772 , 0.9796646 ], dtype=float32)>, <tf.Variable 'pcen/delta:0' shape=(64,) dtype=fl